# <div style="text-align: center;">
  <h1><span style="color:blue; font-family:'Bodoni MT Black';">Web Scraping </span><span style="color:red; font-family:'Bodoni MT Black';">API </span><span style="color:white; font-family:'Bodoni MT Black';">NASA-EONET</span></h1>
</div>


<p>Mediante el uso de aplicaciones cliente, como Worldview de NASA EOSDIS, los usuarios pueden navegar por todo el mundo diariamente y buscar eventos naturales a medida que ocurren. Las tormentas se ven regularmente en los trópicos, tormentas de polvo sobre los desiertos, incendios forestales en los veranos. Estos eventos ocurren constantemente y las imágenes NRT de la NASA pueden representarlos todos utilizando una variedad de parámetros de datos diferentes. Sin embargo, la experiencia del usuario está personalizada y, por lo tanto, restringida por la aplicación del cliente.</p>

<p><span style="color:blue; font-family:'Bodoni MT Black'; font-size: 22px;">¿Qué pasaría si hubiera una API que proporcionara una lista seleccionada de eventos naturales y proporcionara una forma de vincular esos eventos a capas de imágenes NRT relacionadas con eventos? </span> Entra EONET.</p>

<p>El Rastreador de Eventos Naturales del Observatorio de la Tierra (<span style="color:red; font-family:'Bodoni MT Black'; font-size: 22px;">EONET</span>) es un servicio web prototipo con el objetivo de:</p>

<ul>
  <li><span style="color:red; font-family:'Bodoni MT Black'; font-size: 22px;">Proporcionar una fuente seleccionada de metadatos de eventos naturales continuamente actualizados;</span></li>
  <li><span style="color:red; font-family:'Bodoni MT Black'; font-size: 22px;">Proporcionar un servicio que vincule esos eventos naturales con fuentes de imágenes habilitadas para servicios web relacionadas temáticamente(por ejemplo, a través de WMS, WMTS, etc.). </span></ul>
     <p> Consulte la documentación de nuestra API para obtener más información sobre cómo utilizar los servicios web de EONET.</p>


<p>El desarrollo de EONET comenzó en 2015 y ha sido apoyado por el Proyecto del Sistema de Información y Datos de Ciencias de la Tierra y el Observatorio de la Tierra (ESDIS) de la NASA.</p>




# <span style="color:white; font-family:'Bodoni MT Black';">API </span>

In [2]:
# Librerias a utilizar
import pandas as pd
import numpy as np
import matplotlib.pyplot as ptl
import seaborn as sns
from urllib.request import urlopen
from bs4 import BeautifulSoup as soup
import requests
import json
import csv

In [3]:

url="https://eonet.gsfc.nasa.gov/api/v2.1/events"
uclient=urlopen(url)
responsehtml=uclient.read()
uclient.close()
#print(responsehtml.decode())
type(responsehtml.decode())

str

In [11]:
data = responsehtml.decode()
info = json.loads(data)
print(info)

{'title': 'EONET Events', 'description': 'Natural events from EONET.', 'link': 'https://eonet.gsfc.nasa.gov/api/v2.1/events', 'events': [{'id': 'EONET_6367', 'title': 'Typhoon Guchol', 'description': '', 'link': 'https://eonet.gsfc.nasa.gov/api/v2.1/events/EONET_6367', 'categories': [{'id': 10, 'title': 'Severe Storms'}], 'sources': [{'id': 'GDACS', 'url': 'https://www.gdacs.org/report.aspx?eventid=1000974&amp;episodeid=5&amp;eventtype=TC'}, {'id': 'JTWC', 'url': 'https://www.metoc.navy.mil/jtwc/products/wp0323.tcw'}], 'geometries': [{'date': '2023-06-06T12:00:00Z', 'type': 'Point', 'coordinates': [134.8, 14.4]}, {'date': '2023-06-06T18:00:00Z', 'type': 'Point', 'coordinates': [134.2, 14.7]}, {'date': '2023-06-07T00:00:00Z', 'type': 'Point', 'coordinates': [134.1, 14.6]}, {'date': '2023-06-07T06:00:00Z', 'type': 'Point', 'coordinates': [133.8, 15.2]}, {'date': '2023-06-07T12:00:00Z', 'type': 'Point', 'coordinates': [133.1, 15.3]}, {'date': '2023-06-07T18:00:00Z', 'type': 'Point', 'coor

In [5]:
# Convertir los datos JSON en un DataFrame de pandas
df = pd.DataFrame(info['events'])

# Separar los datos en columnas separadas
df['TÍTULO_CATEGORÍA'] = df['categories'].apply(lambda x: x[0]['title'])
df['ID_CATEGORÍA'] = df['categories'].apply(lambda x: x[0]['id'])

df['ID_FUENTE'] = df['sources'].apply(lambda x: x[0]['id'])
df['URL_FUENTE'] = df['sources'].apply(lambda x: x[0]['url'])

df['FECHA_GEOMETRÍA'] = df['geometries'].apply(lambda x: x[0]['date'])
df['TIPO_GEOMETRÍA'] = df['geometries'].apply(lambda x: x[0]['type'])
df['COORDENADAS_GEOMETRÍA'] = df['geometries'].apply(lambda x: [coord['coordinates'] for coord in x])

# Expandir la lista de coordenadas en múltiples filas
df = df.explode('COORDENADAS_GEOMETRÍA')

# Separar las coordenadas en latitud y longitud
df['LATITUD'] = df['COORDENADAS_GEOMETRÍA'].apply(lambda x: x[1])
df['LONGITUD'] = df['COORDENADAS_GEOMETRÍA'].apply(lambda x: x[0])

# Eliminar las columnas 
df.drop(['sources', 'geometries', 'categories','COORDENADAS_GEOMETRÍA'], axis=1, inplace=True)

# Renombrar las columnas
df.rename(columns={
    'id': 'ID',
    'title': 'TÍTULO',
    'description': 'DESCRIPCIÓN',
    'link': 'ENLACE',
    'TÍTULO_CATEGORÍA': 'CATEGORÍA_TÍTULO',
    'ID_CATEGORÍA': 'CATEGORÍA_ID',
    'ID_FUENTE': 'FUENTE_ID',
    'URL_FUENTE': 'FUENTE_URL',
    'FECHA_GEOMETRÍA': 'GEOMETRÍA_FECHA',
    'TIPO_GEOMETRÍA': 'GEOMETRÍA_TIPO',
    'FECHA': 'FECHA',
    'LATITUD': 'LATITUD',
    'LONGITUD': 'LONGITUD'
}, inplace=True)

# Guardar el DataFrame en un archivo CSV
df.to_csv('datoseventos.csv', encoding='UTF-8', index=False, quoting=csv.QUOTE_ALL)


# <span style="color:red; font-family:'Bodoni MT Black';">CATEGORIAS</span>

In [6]:
import json

data = responsehtml.decode()
info = json.loads(data)
#print(info)

In [23]:
# Contar la cantidad de cada categoría
cantidad_categorias = {}
for evento in info["events"]:
    for categoria in evento["categories"]:
        titulo_categoria = categoria["title"]
        if titulo_categoria in cantidad_categorias:
            cantidad_categorias[titulo_categoria] += 1
        else:
            cantidad_categorias[titulo_categoria] = 1

# Imprimir la cantidad y el nombre de cada categoría
for categoria, cantidad in cantidad_categorias.items():
    print("Categoría:", categoria)
    print("Cantidad de eventos:", cantidad)
    print()

Categoría: Severe Storms
Cantidad de eventos: 3

Categoría: Volcanoes
Cantidad de eventos: 26

Categoría: Sea and Lake Ice
Cantidad de eventos: 56

Categoría: Wildfires
Cantidad de eventos: 335



In [29]:
# SEPARAR POR CATEGORIA
TORMENTAS_df= pd.DataFrame()
VOCANES_ACTIVOS_df= pd.DataFrame()
ICEBERG_df= pd.DataFrame()
INCEBDIOS_FORESTALES_df= pd.DataFrame()

for event in info['events']:
    categories = event['categories']
    if any(category['title'] == 'Severe Storms' for category in categories):
        TORMENTAS_df =TORMENTAS_df.append(event, ignore_index=True)
    if any(category['title'] == 'Volcanoes' for category in categories):
        VOCANES_ACTIVOS_df = VOCANES_ACTIVOS_df.append(event, ignore_index=True)
    if any(category['title'] == 'Sea and Lake Ice' for category in categories):
        ICEBERG_df = ICEBERG_df.append(event, ignore_index=True)
    if any(category['title'] == 'Wildfires' for category in categories):
        INCEBDIOS_FORESTALES_df = INCEBDIOS_FORESTALES_df.append(event, ignore_index=True)


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_19044\3064020110.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  TORMENTAS_df =TORMENTAS_df.append(event, ignore_index=True)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_19044\3064020110.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  VOCANES_ACTIVOS_df = VOCANES_ACTIVOS_df.append(event, ignore_index=True)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_19044\3064020110.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  TORMENTAS_df =TORMENTAS_df.append(event, ignore_index=True)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_19044\3064020110.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat in

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_19044\3064020110.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  VOCANES_ACTIVOS_df = VOCANES_ACTIVOS_df.append(event, ignore_index=True)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_19044\3064020110.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  INCEBDIOS_FORESTALES_df = INCEBDIOS_FORESTALES_df.append(event, ignore_index=True)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_19044\3064020110.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  INCEBDIOS_FORESTALES_df = INCEBDIOS_FORESTALES_df.append(event, ignore_index=True)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_19044\3064020110.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pa

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_19044\3064020110.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  INCEBDIOS_FORESTALES_df = INCEBDIOS_FORESTALES_df.append(event, ignore_index=True)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_19044\3064020110.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  INCEBDIOS_FORESTALES_df = INCEBDIOS_FORESTALES_df.append(event, ignore_index=True)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_19044\3064020110.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  INCEBDIOS_FORESTALES_df = INCEBDIOS_FORESTALES_df.append(event, ignore_index=True)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_19044\3064020110.py:16: FutureWarning: The frame.append method is deprecated and will be remov

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_19044\3064020110.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  INCEBDIOS_FORESTALES_df = INCEBDIOS_FORESTALES_df.append(event, ignore_index=True)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_19044\3064020110.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  INCEBDIOS_FORESTALES_df = INCEBDIOS_FORESTALES_df.append(event, ignore_index=True)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_19044\3064020110.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  INCEBDIOS_FORESTALES_df = INCEBDIOS_FORESTALES_df.append(event, ignore_index=True)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_19044\3064020110.py:16: FutureWarning: The frame.append method is deprecated and will be remov

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_19044\3064020110.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ICEBERG_df = ICEBERG_df.append(event, ignore_index=True)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_19044\3064020110.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  VOCANES_ACTIVOS_df = VOCANES_ACTIVOS_df.append(event, ignore_index=True)


In [30]:
#TORMETAS SEVERAS
print(TORMENTAS_df)

           id                       title description  \
0  EONET_6367              Typhoon Guchol               
1  EONET_6366  Tropical Cyclone Biparyjoy               
2  EONET_6363       Tropical Storm Arlene               

                                                link  \
0  https://eonet.gsfc.nasa.gov/api/v2.1/events/EO...   
1  https://eonet.gsfc.nasa.gov/api/v2.1/events/EO...   
2  https://eonet.gsfc.nasa.gov/api/v2.1/events/EO...   

                               categories  \
0  [{'id': 10, 'title': 'Severe Storms'}]   
1  [{'id': 10, 'title': 'Severe Storms'}]   
2  [{'id': 10, 'title': 'Severe Storms'}]   

                                             sources  \
0  [{'id': 'GDACS', 'url': 'https://www.gdacs.org...   
1  [{'id': 'GDACS', 'url': 'https://www.gdacs.org...   
2  [{'id': 'NOAA_NHC', 'url': 'https://www.nhc.no...   

                                          geometries  
0  [{'date': '2023-06-06T12:00:00Z', 'type': 'Poi...  
1  [{'date': '2023-06-05T18:00

In [31]:
#VOLCANES ACTIVOS
print(VOCANES_ACTIVOS_df)

            id                                   title description  \
0   EONET_6369                  Ibu Volcano, Indonesia               
1   EONET_6352       Kadovar Volcano, Papua New Guinea               
2   EONET_6348            Barren Island Volcano, India               
3   EONET_6345  Rincon de la Vieja Volcano, Costa Rica               
4   EONET_6342               Dukono Volcano, Indonesia               
5   EONET_6336              Bezymianny Volcano, Russia               
6   EONET_6337                Kita-loto Volcano, Japan               
7   EONET_6333           Semisopochnoi Volcano, Alaska               
8   EONET_6331                  Ambae Volcano, Vanuatu               
9   EONET_6326          Karangetang Volcano, Indonesia               
10  EONET_6319            Lewotolok Volcano, Indonesia               
11  EONET_6309       Nevado del Ruiz Volcano, Colombia               
12  EONET_6296             Krakatau Volcano, Indonesia               
13  EONET_6291      

In [32]:
#ICEBERG
print(ICEBERG_df)

            id          title                            description  \
0   EONET_6354   Iceberg A76E                                          
1   EONET_6355   Iceberg A76F                                          
2   EONET_6356   Iceberg A76G                                          
3   EONET_6357   Iceberg A76H                                          
4   EONET_6358   Iceberg A76J                                          
5   EONET_6359   Iceberg A76K                                          
6   EONET_6360   Iceberg A76L                                          
7   EONET_6347   Iceberg A76D                                          
8   EONET_6344    Iceberg C40                                          
9   EONET_6330   Iceberg A80D                                          
10  EONET_6320    Iceberg A81                                          
11  EONET_6302   Iceberg A80A                                          
12  EONET_6288    Iceberg D32                                   

In [33]:
#INCENDIOS FORESTALES
print(INCEBDIOS_FORESTALES_df)

             id                                              title  \
0    EONET_6350  Major Fire, Ocala National Forest, Florida, Un...   
1    EONET_6343                                    Great Lake Fire   
2    EONET_6280                                   Nakia Creek Fire   
3    EONET_6269                                         Bovee Fire   
4    EONET_6283                                       Siouxon Fire   
..          ...                                                ...   
330  EONET_5405  Wildfire - NW of Powell (Robertson Draw Fire),...   
331  EONET_5393  Wildfires - Graham County (Pinnacle Fire), Ari...   
332  EONET_5374  Wildfire - Pinal and Gila Counties (Mescal Fir...   
333  EONET_5337                Wildfire MB-EA047, Manitoba, Canada   
334  EONET_5329                Wildfire MB-EA044, Manitoba, Canada   

    description                                               link  \
0                https://eonet.gsfc.nasa.gov/api/v2.1/events/EO...   
1                ht

# <span style="color:blue; font-family:'Bodoni MT Black';">MAPAS</span>